## Working through an example case for a player

In [1]:
import requests
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)

In [5]:
#put in the api key
api_key = 'RGAPI-314476db-1fb1-4941-b92f-fe19912e2256'

#make a request to grab the challenger players from NA 
#this is top 200 players (# len(challenger_players.json()['entries']))
challenger_players = requests.get(
    'https://na1.api.riotgames.com/tft/league/v1/challenger' #gets all NA challenger players
    +'?api_key=' + api_key)

In [55]:
#here is a sample player from the response
player_summoner_name = challenger_players.json()['entries'][0]['summonerName']

player_summoner_name

'Kira'

In [75]:
player_summoner_name = 'Harka'

In [76]:
#we grab the sample player's account info
player_account_info = requests.get(
    'https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/' +
    player_summoner_name
    +'?api_key=' + api_key)   

player_account_info.json()

{'accountId': 'eXpqYoJ25sRRtsDtCotRyCYNMoFPf99P-oqjr0zoUgh0ukFa85I6XqNS',
 'id': 'hzBLaMTBMckhPFR2lif_llJio6jfu2hV2mD1uMYBfPZ9ud7b',
 'name': 'Harka',
 'profileIconId': 29,
 'puuid': 'I6_rHk6KIxKULscewvEmcGU4jwaKQb--c0xOiSM94KlVBjSdyj8Nr38YBxZYAfWMM1_TTvOnOwBJ7w',
 'revisionDate': 1580854524000,
 'summonerLevel': 1}

In [77]:
#we isolate the puuid of the player from the account info
player_puuid = player_account_info.json()['puuid']

player_puuid

'I6_rHk6KIxKULscewvEmcGU4jwaKQb--c0xOiSM94KlVBjSdyj8Nr38YBxZYAfWMM1_TTvOnOwBJ7w'

In [41]:
#make a set of matches so no possible duplicates
matches_set = set()

#we grab the last 20 matches of the player via the player_puuid
player_matches = requests.get(
    'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/'
    + player_puuid
    +'/ids?api_key=' + api_key)

#add the last 20 matches to the set
for match in player_matches.json():
    matches_set.add(match)

In [45]:
#turn into a list so we can iterate through it
matches_set = list(matches_set)
matches_set.sort()
# matches_set

### Info below pertains to a specific match

In [14]:
#we pick a match out of the matches given to us
match_number = player_matches.json()[0]

In [15]:
#we get the info of the match chosen
player_match_info = requests.get(
    'https://americas.api.riotgames.com/tft/match/v1/matches/'
    + match_number
    +'?api_key=' + api_key
)

In [16]:
#create a new dataframe to house the data
data = pd.DataFrame()

In [17]:
#we isolate only on the 'info/participants' section since it's all we care about
match = player_match_info.json()['info']['participants']

for player in match:
    
#     players_data = dict.fromkeys(list_of_tft_units,0) #create a dict to insert data into 
    players_data = {}
    
    #capture the units
    units = player['units'] 
    for unit in units:
        unit_name = unit['character_id'].replace('TFT2_','').lower()
        unit_tier = unit['tier']
        players_data[unit_name] = unit_tier  
    
    #capture the traits
    traits = player['traits']

    for trait in traits:
        trait_name = '_trait_' + trait['name'].lower()
        tier = trait['tier_current']
        players_data[trait_name] = tier 
        
    #capture the placement
    players_data['_placement'] = player['placement'] 

    #capture the level
    players_data['_level'] = player['level']

    #add this all to the dataframe
    data = data.append(players_data, ignore_index=True)

In [115]:
data = data.fillna(0) #fill out the zeroes
data = data.sort_index(axis=1) #re-sort the index

move_to_front = ['_placement','_level'] #move placement and level to the front
data = data[move_to_front + [col for col in data.columns if col not in move_to_front]]

data = data.astype(int) #turn to whole numbers, should not be any floats

# data

## TABLES OF INFO

### List of all the available TFT Units

In [293]:
# #this is the list of all available units in the game
# list_of_tft_units = [
#     "Aatrox", "Amumu", "Annie", "Ashe", "Azir" #a
#     ,"Brand","Braum" #b
#     ,"Diana","Dr. Mundo" #d
#     ,"Ezreal" #e
#     ,"Ivern" #i
#     ,"Janna","Jax" #j
#     ,"Karma","Kha'Zix","Kindred","KogMaw" #k
#     ,"LeBlanc","Leona","Lucian"#l
# #     ,"LuxCloud","LuxCrystal","LuxElectric","LuxGlacial","LuxInferno" #lux
#     ,"LuxWind","LuxCrystal","LuxElectric","LuxGlacial","LuxInferno" #lux    
#     ,"LuxLight","LuxOcean","LuxShadow","LuxWoodland" #lux
#     ,"Malphite","Malzahar","Maokai","Master Yi" #m
#     ,"Nami","Nasus","Nautilus","Neeko","Nocturne" #n
#     ,"Olaf","Ornn" #o
#     ,"QiyanaCloud","QiyanaInferno","QiyanaMountain","QiyanaOcean" #qiyana
#     ,"RekSai","Renekton" #r
#     ,"Senna","Singed","Sion","Sivir","Skarner","Soraka","Syndra" #s
#     ,"Taliyah","Taric","Thresh","Twitch" #t
#     ,"Varus","Vayne","Veigar","Vladimir","Volibear" #v
#     ,"Warwick" #w
#     ,"Yasuo", "Yorick" #y
#     ,"Zed","Zyra" #z
# ]

# #strip special characters, replace spaces and turn to all lower case
# list_of_tft_units = [re.sub('[^A-Za-z0-9]+', '', name) for name in list_of_tft_units]
# list_of_tft_units = [name.replace(' ', '') for name in list_of_tft_units]
# list_of_tft_units = [name.lower() for name in list_of_tft_units]

# ##CONSIDER THAT CLOUD IS CALLED 'WIND' FROM ACTUAL TFT API

# ACTUAL RUNS

## Script to run to get matches

In [3]:
import requests
import pandas as pd
import numpy as np
import re
import time

pd.set_option('display.max_columns', 500)

start_time = time.time()

#######################################################################################
api_key = 'RGAPI-314476db-1fb1-4941-b92f-fe19912e2256' #put in the api key
#######################################################################################

matches_set = set()             #make a set of matches so no possible duplicates
data = pd.DataFrame()           #create a new dataframe to house the data
list_of_challenger_players = [] #make a list of challenger players

#make a request to grab the challenger players from NA 
#this is 200 players (len(challenger_players.json()['entries']))
challenger_players = requests.get(
    'https://na1.api.riotgames.com/tft/league/v1/challenger' 
    +'?api_key=' + api_key)

#put into list of challenger players
for player in challenger_players.json()['entries']:
    list_of_challenger_players.append(player['summonerName'])

#iterate through each challenger player
for i in range(0, len(list_of_challenger_players)):
    player_summoner_name = list_of_challenger_players[i]

    #we grab the player's account info
    player_account_info = requests.get(
        'https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/' +
        player_summoner_name
        +'?api_key=' + api_key)   

    #we isolate the puuid of the player from the account info
    player_puuid = player_account_info.json()['puuid']

    #we grab the last 20 matches of the player via the player_puuid
    player_matches = requests.get(
        'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/'
        + player_puuid
        +'/ids?api_key=' + api_key)

    #add the last 20 matches to the set
    for match in player_matches.json():
        matches_set.add(match)
        
    time.sleep(1)

    
# #turn into a list so we can iterate through it
matches_set = list(matches_set)
matches_set.sort()

print("Time for grabbing all matches:", time.time() - start_time)

Time for grabbing all matches: 316.98940205574036


## Script to pull data from matches into df

## Testing on first 500 of 2379 matches found

In [5]:
temp_set = matches_set[0:500]

In [6]:
start_time = time.time()

for match_number in temp_set:
# for match_number in matches_set:

    #we get the info of the match chosen
    player_match_info = requests.get(
        'https://americas.api.riotgames.com/tft/match/v1/matches/'
        + match_number
        +'?api_key=' + api_key
    )

    #we isolate only on the 'info/participants' section since it's all we care about
    match = player_match_info.json()['info']['participants']

    for player in match:

        players_data = {}

        #capture the units
        units = player['units'] 
        for unit in units:
            unit_name = unit['character_id'].replace('TFT2_','').lower()
            unit_tier = unit['tier']
            players_data[unit_name] = unit_tier  

        #capture the traits
        traits = player['traits']

        for trait in traits:
            trait_name = '_trait_' + trait['name'].lower()
            tier = trait['tier_current']
            players_data[trait_name] = tier 

        #capture the placement
        players_data['_placement'] = player['placement'] 

        #capture the level
        players_data['_level'] = player['level']

        #add this all to the dataframe
        data = data.append(players_data, ignore_index=True)
        
       
    time.sleep(1)
    
    
data = data.fillna(0) #fill out the zeroes
data = data.sort_index(axis=1) #re-sort the index

move_to_front = ['_placement','_level'] #move placement and level to the front
data = data[move_to_front + [col for col in data.columns if col not in move_to_front]]

data = data.astype(int) #turn to whole numbers, should not be any floats

print("Time for grabbing all data from matches:", time.time() - start_time)

Time for grabbing all data from matches: 664.8944489955902


## Testing Models

In [111]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [66]:
data_y = data['_placement']
data_x = data[data.columns[~data.columns.isin(['_placement'])]]

x_train, x_test, y_train, y_test = train_test_split(data_x,data_y, test_size = 0.2)

In [114]:
example = [data[data.columns[~data.columns.isin(['_placement'])]].iloc[1]]

#linear regression model
lin_reg_model = LinearRegression().fit(x_train,y_train)
print(lin_reg_model.predict(example))

#logistic regression model
log_reg_model = LogisticRegression().fit(x_train,y_train)
print(log_reg_model.predict(example))

#naive bayes model
nb_model = GaussianNB().fit(x_train,y_train)
print(nb_model.predict(example))

[4.78687248]
[6]
[7]


In [103]:
data.head()

,_placement,_level,_trait_alchemist,_trait_avatar,_trait_berserker,_trait_celestial,_trait_crystal,_trait_desert,_trait_druid,_trait_electric,_trait_inferno,_trait_light,_trait_mage,_trait_metal,_trait_mountain,_trait_mystic,_trait_ocean,_trait_poison,_trait_predator,_trait_set2_assassin,_trait_set2_blademaster,_trait_set2_glacial,_trait_set2_ranger,_trait_shadow,_trait_soulbound,_trait_summoner,_trait_warden,_trait_wind,_trait_woodland,aatrox,amumu,annie,ashe,azir,brand,braum,diana,drmundo,ezreal,ivern,janna,jax,karma,khazix,kindred,kogmaw,leblanc,leona,lucian,luxcrystal,luxelectric,luxglacial,luxinferno,luxlight,luxmetal,luxocean,luxshadow,luxwind,luxwoodland,malphite,malzahar,maokai,masteryi,nami,nasus,nautilus,neeko,nocturne,olaf,ornn,qiyanainferno,qiyanaocean,qiyanawind,qiyanawoodland,reksai,renekton,senna,singed,sion,sivir,skarner,soraka,syndra,taliyah,taric,thresh,twitch,varus,vayne,veigar,vladimir,volibear,warwick,yasuo,yorick,zed,zyra
0,2,9,1,1,2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,2,2,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
9,4,8,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,1,1,0,2,0,0,0,2,0,0,0,0,0,0,2,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
10,7,7,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,1,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
11,2,8,0,1,0,0,0,0,0,0,2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,5,8,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,2,0,1


In [ ]:
'''
current concerns:
    1. having the level of a unit may be too granular of data 
    2. able to get the current top team compositions from other websites, but there is no historical
    3. is it better to display confidence intervals?
    
thoughts on current models:    
    logistic regression vs naive bayes 
        naive bayes assumes all features to be independent, so no similarity
        
general questions:
    1. what is the best way to check similarity/no relationship between features?
'''